In [1]:
import math
import numpy as np
import pandas as pd
from scipy.special import logsumexp

### Functions that will be called in the main code

In [2]:
def fitness(init_pop, n, dimension, upper_b, lower_b, fobj):
    fit1 = np.zeros((n,1))

    for i in range(len(init_pop), 1):
        fit1[i] = fobj[init_pop[i, :]]
    
    #best fit and position
    best_fit = fit[0]
    pos = 0
    for i in range(1, len(fit1)):
        if best_fit > fit1[i]:
            best_fit = fit1[i]
            pos = i


    return fit1, best_fit, pos

In [3]:
def herding(pop, Vt, fit, n, dimension, acc, time):
    fit1 = np.sort(fit)
    idx = np.sort(fit)
    pop1 = np.zeros((n, dimension))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))
    
    for i in range(1, n):
        pop1[i, :] = pop[int(idx[i]), :]
        Vt1[i, :] = Vt[int(idx[i]), :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return pop1, Vt1, fit1, acc1, time1

In [23]:
def generate(n, dimension, upper_b, lower_b):
    boundary_no = 1
    pop = np.zeros((n, dimension))
    
    if boundary_no == 1:
        #Analisar esse rand
        pop = np.random.random(size=(n, dimension))*((upper_b - lower_b) + lower_b)
    
    #Unused part of the algorithm, that's why it's commented
    #if boundary_no > 1:
    #    for i in range(1,dimension):
    #        pop = np.random.random(size = (n,1))*(upper_b-lower_b)+lower_b;
    x = np.random.random(size = (n, dimension))
    M = pop

    return M, x


In [16]:
def update(pop, Vt, time, acc, n, dimension, eye):
    pop1 = np.zeros((n,dimension))
    for i in range(1, n):
        for j in range(1, dimension):
            #Updating the position of dogs
            if (i <= 3):
                pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
            #Updating position of sheep
            if (i > 3):
                if eye == 1:
                    pop1[i][j] = Vt[i][j] * time[i] - (1/2) * acc[i][j] * (np.power(time[i], 2))
                else:
                    pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
    return pop1

In [17]:
def update_v(Vt, n, dimension, acc, time, pop, fit, eye):
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = Vt
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[0]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[2] + fit[3]) / 2
    f = 0
    tempg = [0 for i in range(n)]
    temps = [0 for i in range(n)]

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(1, n):
        for j in range(1, dimension):
            #Velocity updation of dogs
            if (i <= 3):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * (pop[i][j]))

            #Velocity updation of sheep
            if (i > 3):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc[f][j]) * pop[i][j])

                else:
                    #Velocity updation of gathered sheep
                    if (fit1 - fit[i] > fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if (fit1 - fit[i] <= fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*pop[right_dog][j])) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*pop[left_dog][j]))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        temps[i] = i
    
    #Updation of time and acceleration
    for i in range(1, n):
        s = 0
        for j in range(1, dimension):
            acc[i] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i])
            s += ((Vt1[i][j]) - (Vt[i][j])) / (acc[i][j])
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [18]:
def check(pop, n, dimension, upper_b, lower_b, acc, Vt, time):
    pop1 = pop
    acc1 = acc
    time1 = time
    Vt1 = Vt
    for i in range(n):
        for j in range(dimension):

            if (pop[i][j] >= upper_b or pop[i][j] <= lower_b or pop[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1

In [19]:
def Ufun(x, a, k, m):
        op = k*((x-a)^m) * (x>a) + k * ((-x-a)^m) * (x < (-a))
        
        return op

class functions:
    def func_details(self, fn):
        default = "Invalid function!!!"

        return getattr(self, 'case_'+str(fn), lambda: default)()

    #Funcionando
    def func1(self, x):
        lower_b = -100
        upper_b = 100
        dimension = 30

        op = np.sum(x^2)

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func2(self, x):
        lower_b = -10
        upper_b = 10
        dimension = 30

        op = np.abs(x) + np.abs(x)

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func3(self, x):
        lower_b = -100
        upper_b = 100
        dimension = 30

        dim = x
        op = 0
        for i in range(dim):
            op += x^2

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func4(self, x):
        lower_b = -100
        upper_b = 100
        dimension = 30

        op = x

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func5(self, x):
        lower_b = -30
        upper_b = 30
        dimension = 30

        dim = x
        op = (100*x) - (x^2)^2 + (x-1)^2

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func6(self, x):
        lower_b = -100
        upper_b = 100
        dimension = 30

        op = np.power(abs(x+0.5), 2)

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func7(self, x):
        lower_b = -1.28
        upper_b = 1.28
        dimension = 30

        dim = x
        op = np.sum(x * (x^4)) + np.random.random()

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func8(self, x):
        lower_b = -500
        upper_b = 500
        dimension = 30

        op = np.sum(-x * math.sin(math.sqrt(abs(x))))

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func9(self, x):
        lower_b = -32
        upper_b = 32
        dimension = 30

        dim = x
        op = -20 * math.exp(-0.2 * math.sqrt(int(x^2)/dim)) - math.exp(math.cos(2 * math.pi * x)/dim) + 20 + math.exp(1)

        return lower_b, upper_b, dimension, op

    #Consertar
    def func10(self, x):
        lower_b = -50
        upper_b = 50
        dimension = 30

        dim = x
        op = (math.pi/dim) * (10* (math.sin(math.pi * 1 + (x + 1)/4))^2) + np.sum(((((x+1)/4)^2)) * (1+10 * ((math.sin(math.pi * float((1+(x+1)/4)))^2))) + float(((x+1)/4)^2)) + np.sum(Ufun(x, 10, 100, 4))

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func11(self, x):
        lower_b = -50
        upper_b = 50
        dimension = 30

        dim = x
        op = 0.1 * (np.power((math.sin(3*math.pi*x)), 2) + np.sum(np.power((x-1), 2) * np.power(1+(math.sin(3*math.pi*x)), 2)) + np.power((x-1), 2)* np.power(1+(math.sin(2*math.pi*x)), 2)) + np.sum(Ufun(x,5,100,4))

        return lower_b, upper_b, dimension, op

    #Consertar
    def func12(self, x):
        lower_b = -65.536
        upper_b = 65.536
        dimension = 2

        aS = [[-32, -16, 0, 16, 32, -32, -16, 0, 16, 32, -32, -16, 0, 16, 32, -32, -16, 0, 16, 32, -32, -16, 0, 16, 32],
        -32, -32, -32, -32, -32, -16, -16, -16, -16, -16, 0, 0, 0, 0, 0, 16, 16, 16, 16, 16, 32, 32, 32, 32, 32]
        bS = [0 for i in range(25)]
        
        for j in range(1, 25):
            #Rever o codigo para ver a necessidade de transpor a matriz
            bS[j] = np.sum((x - aS[:,j])^6)

        op = (1/500 + np.sum(1/(bS)))^(-1)

        return lower_b, upper_b, dimension, op

    #Consertar
    def func13(self, x):
        lower_b = -5
        upper_b = 5
        dimension = 4

        aK = [.1957, .1947, .1735, .16, .0844, .0627, .0456, .0342, .0323, .0235, .0246]
        bK = [.25, .5, 1, 2, 4, 6, 8, 10, 12, 14, 16]
        
        for i in range(len(bK)):
            bK[i] = 1/bK[i]
            op = np.sum((aK[i]-((x * (bK[i]^2+x*bK[i]))/(bK[i]^2+x * bK[i]+x)))^2)

        return lower_b, upper_b, dimension, op

    #Funcionando
    def func14(self, x):
        lower_b = -2
        upper_b = 2
        dimension = 2

        op = (1 + (x + x+1)^2 * (19-14*x+3*(x^2)-14*x+6*x*x+3*x^2)) * (30+(2*x-3*x)^2*(18-32*x+12*(x^2)+48*x-36*x*x+27*(x^2)))

        return lower_b, upper_b, dimension, op

    #Ate a func19 problema com slice, int recebe list
    def func15(self, x):
        lower_b = 0
        upper_b = 1
        dimension = 3

        aH = [[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]]
        cH = [1, 1.2, 3, 3.2]
        pH = [[0.3689, 0.117, 0.2673], [0.4699, 0.4387, 0.747], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]]
        op = 0

        for i in range(1,4):
            op = op - cH[i]*math.exp(-(np.sum(aH[i:]*((x-pH[i:])^2))))

        return lower_b, upper_b, dimension, op

    def func16(self, x):
        lower_b = 0
        upper_b = 1
        dimension = 6

        aH = [[10, 3, 17, 3.5, 1.7, 8], [0.05, 10, 17, 0.1, 8, 14], [3, 3.5, 1.7, 10, 17, 8], [17, 8, 0.05, 10, 0.1, 14]]
        cH = [1, 1.2, 3, 3.2]
        pH = [[0.1312, 0.1696, 0.5569, 0.0124, 0.8283, 0.5886], [0.2329, 0.4135, 0.8307, 0.3736, 0.1004, 0.9991], [0.2348, 0.1415, 0.3522, 0.2883, 0.3047, 0.6650], [0.4047, 0.8828, 0.8732, 0.5743, 0.1091, 0.0381]]
        op = 0

        for i in range(1,4):
            op = op - cH[i]*math.exp(-(np.sum(aH[i,:]* ((x-pH[1,:])^2))))

        return lower_b, upper_b, dimension, op

    def func17(self, x):
        lower_b = 0
        upper_b = 10
        dimension = 4

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0

        for i in range(1,5):
            #Apostrofo
            op = op - ((x-aSH[1,:])*(x-aSH[i,:])+cSH[i])^(-1)

        return lower_b, upper_b, dimension, op

    def func18(self, x):
        lower_b = 0
        upper_b = 10
        dimension = 4

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0
        for i in range(1, 7):
            #Apostrofo
            op = op - ((x - aSH[i,:])*(x-aSH[i,:])+cSH[i])^(-1)

        return lower_b, upper_b, dimension, op

    def func19(self, x):
        lower_b = 0
        upper_b = 10
        dimension = 4

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0

        for i in range(1, 10):
            #Apostrofo
            op = op- ((x-aSH[i,:])*(x-aSH[i,:]) + cSH[i])^(-1)

        return lower_b, upper_b, dimension, op
    

### Main code

In [22]:
#Create class
collie = functions()

#Population size
n = 30

#Maximum no. of iterations
gen = 200

#Optimization function name
lower_b, upper_b, dim, op = collie.func1(n)

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate(n, dim, upper_b, lower_b)
#Vt = velocity of each individuals
Vt = np.zeros((n, dim))
#Time of each individual
time = np.random.random(size = (n,1))

#Max fitness value
fopt = math.inf

#Variable to store fitness
fit = np.zeros((gen, 1))

pop = init_p

#k = counter variable for iterations  required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))

for g in range(gen):
    #Calculate fitness of indivuals
    #Olhar o fitness tb
    fit, maxf, pos = fitness(pop, n, dim, upper_b, lower_b, op)
    
    eye = 0
    if g == 1:
        fopt = maxf
    
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf

    fopt_1[g] = fopt

    if g > 1:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    pop, Vt, fit, acc, time = herding(pop, Vt, fit, n, dim, acc, time)
    #Funcionando
    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, pop, fit, eye)
    #Funcionando
    pop = update(pop, Vt, time, acc, n, dim, eye)
    #Funcionando
    pop, acc, time, Vt = check(pop, n, dim, upper_b, lower_b, acc, Vt, time)

    print("\n\n fopt  %.4f\t maxf %.4f\t gen %d"%(fopt, maxf, g))


Init p ->  [[23.90152097 50.01407111 19.61300641  9.03001638 89.3246585  87.50030624
  33.08985598 50.10964404  2.08896993 80.61003974 51.07290294 42.99400297
  19.69642946 42.63233011 13.81215086 98.67527612 99.66175407  8.80896795
  16.06419758 51.48758307 91.23168966 80.0345528  82.21072678 48.12465607
  41.83690509 14.03427529 25.83565879 33.01226721 45.55887302 94.57979607]
 [87.3164009   5.20419885 13.62576759 69.16489515 53.60311162 81.6340409
  19.40968408 87.08087359 56.35900162 65.89799501 35.50909691 31.4694833
  17.10965606 84.59711491 23.01177184  9.52348371 48.49757619 27.16962166
  38.65342834 47.67916973 70.24726303 43.9291794  26.5398294  11.96681288
  14.49169237 61.48092887 42.47237378 45.96260847 60.11395492 56.74071199]
 [54.290194   26.10485439 48.11159832 42.39158344 83.6948036  98.38756455
  61.60548995 25.97903996 88.36337279 60.36392358 91.07006442 95.14904494
  16.50799169 74.50969614 73.44247596 86.25655779 51.9690953  98.56901694
  31.18722506 41.49412685 7

C:\Users\artur\AppData\Local\Temp/ipykernel_9820/2658963210.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += ((Vt1[i][j]) - (Vt[i][j])) / (acc[i][j])




 fopt  0.0000	 maxf 0.0000	 gen 7


 fopt  0.0000	 maxf 0.0000	 gen 8


 fopt  0.0000	 maxf 0.0000	 gen 9


 fopt  0.0000	 maxf 0.0000	 gen 10


 fopt  0.0000	 maxf 0.0000	 gen 11


 fopt  0.0000	 maxf 0.0000	 gen 12


 fopt  0.0000	 maxf 0.0000	 gen 13


 fopt  0.0000	 maxf 0.0000	 gen 14


 fopt  0.0000	 maxf 0.0000	 gen 15


 fopt  0.0000	 maxf 0.0000	 gen 16


 fopt  0.0000	 maxf 0.0000	 gen 17


 fopt  0.0000	 maxf 0.0000	 gen 18


 fopt  0.0000	 maxf 0.0000	 gen 19


 fopt  0.0000	 maxf 0.0000	 gen 20


 fopt  0.0000	 maxf 0.0000	 gen 21


 fopt  0.0000	 maxf 0.0000	 gen 22


 fopt  0.0000	 maxf 0.0000	 gen 23


 fopt  0.0000	 maxf 0.0000	 gen 24


 fopt  0.0000	 maxf 0.0000	 gen 25


 fopt  0.0000	 maxf 0.0000	 gen 26


 fopt  0.0000	 maxf 0.0000	 gen 27


 fopt  0.0000	 maxf 0.0000	 gen 28


 fopt  0.0000	 maxf 0.0000	 gen 29


 fopt  0.0000	 maxf 0.0000	 gen 30


 fopt  0.0000	 maxf 0.0000	 gen 31


 fopt  0.0000	 maxf 0.0000	 gen 32


 fopt  0.0000	 maxf 0.0000	 gen 33


 f